In [516]:
# !pip install boto3 sagemaker langchain langchain-community langchain-core faiss-cpu requests opensearch-py sentence-transformers langchain-text-splitters requests-aws4auth qdrant-client -U

In [517]:
# Import  modules and libraries 
import json
import boto3
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap

In [518]:
# Constants 
MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"

In [519]:
bedrock_client = boto3.client('bedrock-runtime', region_name="us-east-1")

In [520]:
# TODO : Include Test API Call into this notebook 

In [521]:
baseline_user_prompt = """
### Here is a user prompt:
{query}
"""

In [522]:
def process_prompt(query_args):
    prompt_with_query = baseline_user_prompt.replace("{query}", query_args['query'])
    
    # This format doesn't matter much now, but we will use it later to 
    # persist chat history for continuous dialogue
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_with_query
                }
            ]
        }
    ]
    
    return messages

In [523]:
# Eval LLM 
def query_eval_llm(messages):
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'messages': messages,
            'max_tokens': 3000,
            'temperature': 0.1, 
            'top_p': 0.9,
            'top_k' : 2,
        })
    )
    response_body = json.loads(response.get('body').read())
    
    return response_body

In [524]:
qdrant_rag_chain_eval = (
    RunnableMap(
         {"query": RunnablePassthrough()}
    )
    | process_prompt
    | query_eval_llm
)

In [525]:
# Import S3 bucket JSON file.

s3 = boto3.client('s3')
bucket_name = 'test-api-results-2'
# file_key = 'test_for_eval_pipeline'
# # Download it to Thomas' local laptop
# s3.download_file(bucket_name, file_key, f"/Users/tl164/Downloads/{file_key}")
# # Download it to Sagemaker instance 
# # s3.download_file(bucket_name, file_key, f'../data/{file_key}')


# response_df = pd.read_json(f'/Users/tl164/Downloads/{file_key}')

# response_df = response_df.transpose()

In [526]:
# import os
# def download_all_objects(bucket_name, download_path):
#     s3 = boto3.client('s3')

#     # List all objects in the bucket
#     objects = s3.list_objects_v2(Bucket=bucket_name)

#     if 'Contents' not in objects:
#         print(f"No objects found in bucket {bucket_name}")
#         return

#     for obj in objects['Contents']:
#         key = obj['Key']
#         file_path = os.path.join(download_path, key)
#         os.makedirs(os.path.dirname(file_path), exist_ok=True)
#         s3.download_file(bucket_name, key, file_path)
#         print(f"Downloaded {key} to {file_path}")

In [527]:
    # download_all_objects('test-api-results-2', f"/Users/tl164/Downloads/final_test_output_no_duplicates")


In [529]:
from pandas import json_normalize

folder_path = f"/Users/tl164/Downloads/final_test_output_no_duplicates"
data_frames = []

# List all files in the folder
for filename in os.listdir(folder_path):
    if filename.startswith('test_queries_results_gatekeeper_False') and filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        output = pd.DataFrame()

        # Read the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)
            
            # Convert JSON data to a DataFrame
            for key in data:
                
                output = output.append(data[key], ignore_index=True)
            data_frames.append(output)

# Concatenate all DataFrames
if data_frames:
    normal_recipe_df = pd.concat(data_frames)
else:
    print("No JSON files found.")


/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/158810966.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(data[key], ignore_index=True)
/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/158810966.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(data[key], ignore_index=True)
/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/158810966.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(data[key], ignore_index=True)
/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/158810966.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Us

In [530]:
from pandas import json_normalize

folder_path = f"/Users/tl164/Downloads/final_test_output_no_duplicates"
data_frames = []

# List all files in the folder
for filename in os.listdir(folder_path):
    if filename.startswith('test_queries_results_gatekeeper_True') and filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        output = pd.DataFrame()

        # Read the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)
            # Convert JSON data to a DataFrame
            for key in data:
                output = output.append(data[key], ignore_index=True)
            data_frames.append(output)

# Concatenate all DataFrames
if data_frames:
    gate_keeper_receipe_df = pd.concat(data_frames)
else:
    print("No JSON files found.")


/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/1564672650.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(data[key], ignore_index=True)
/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/1564672650.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(data[key], ignore_index=True)
/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/1564672650.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(data[key], ignore_index=True)
/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/1564672650.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

In [531]:
normal_recipe_df.reset_index(drop=True, inplace=True)
gate_keeper_receipe_df.reset_index(drop=True, inplace=True)

In [532]:
normal_recipe_df

,Query_Question_No,Query_Question,top_p,top_k,temperature,max_tokens,system_prompt,retriever,coarse_search_type,coarse_top_k,coarse_lambda,reranker_top_n,coarse_fetch_k,self_query_api,self_query_model,Query_Response
0,1,I enjoy asian fusion food and I am a vegetaria...,0.1,10,0.1,3000,\nYou are a helpful assistant and expert in co...,coarse,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,<search_quality_reflection>\nThe additional se...
1,3,I have a peanut allergy but I like thai food. ...,0.1,10,0.1,3000,\nYou are a helpful assistant and expert in co...,coarse,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,<search_quality_reflection>\nThe web search re...
2,4,Suggest a low-carb breakfast recipe that inclu...,0.1,10,0.1,3000,\nYou are a helpful assistant and expert in co...,coarse,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,<search_quality_reflection>\nThe additional Go...
3,5,Suggest a healthy dinner recipe for two people...,0.1,10,0.1,3000,\nYou are a helpful assistant and expert in co...,coarse,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,<search_quality_reflection>\nThe web search re...
4,6,I am on a ketogenic diet and need a dinner rec...,0.1,10,0.1,3000,\nYou are a helpful assistant and expert in co...,coarse,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,<search_quality_reflection>\nThe additional Go...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,12,I am on a heart-healthy diet and need a lunch ...,0.1,2,1.0,3000,\nYou are a helpful assistant and expert in co...,coarse,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,The Heart Healthy Personal Pizza recipe looks ...
536,13,I am allergic to shellfish and soy. I need a v...,0.1,2,1.0,3000,\nYou are a helpful assistant and expert in co...,coarse,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,<search_quality_reflection>\nThe additional Go...
537,14,I am lactose intolerant and need a dessert rec...,0.1,2,1.0,3000,\nYou are a helpful assistant and expert in co...,coarse,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,<search_quality_reflection>\nThe additional we...
538,15,"Suggest a high-fiber, low-calorie lunch recipe...",0.1,2,1.0,3000,\nYou are a helpful assistant and expert in co...,coarse,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,<search_quality_reflection>\nThe additional Go...


In [533]:
gate_keeper_receipe_df

,Query_Question_No,Query_Question,top_p,top_k,temperature,max_tokens,system_prompt,retriever,coarse_search_type,coarse_top_k,coarse_lambda,reranker_top_n,coarse_fetch_k,self_query_api,self_query_model,Query_Response
0,1,I am allergic to peanuts. Can you suggest a de...,0.9,10,1.0,3000,\nYou are a helpful assistant and expert in co...,self_query_chain,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,Here is a delicious nut-free dessert recipe fo...
1,3,I have a shellfish allergy. Please provide a d...,0.9,10,1.0,3000,\nYou are a helpful assistant and expert in co...,self_query_chain,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,"The query retrieved a delicious ""No Shells Cio..."
2,4,I follow a vegan diet. Can you give me a break...,0.9,10,1.0,3000,\nYou are a helpful assistant and expert in co...,self_query_chain,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,Here is a delicious vegan breakfast recipe for...
3,5,I need a gluten-free lunch recipe. What can yo...,0.9,10,1.0,3000,\nYou are a helpful assistant and expert in co...,self_query_chain,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,Here is a delicious gluten-free lunch recipe f...
4,6,I don't eat dairy products. Can you provide a ...,0.9,10,1.0,3000,\nYou are a helpful assistant and expert in co...,self_query_chain,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,Based on the recipe results from the vector da...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,21,"I am looking for a recipe for a cake, but it s...",0.1,2,1.0,3000,\nYou are a helpful assistant and expert in co...,self_query_chain,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,Here is a recipe for a delicious sugar-free ch...
860,22,I am allergic to peanuts. Can you suggest a pe...,0.1,2,1.0,3000,\nYou are a helpful assistant and expert in co...,self_query_chain,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,<search_quality_reflection>\nThe web search re...
861,23,I follow a vegan diet but also want a recipe w...,0.1,2,1.0,3000,\nYou are a helpful assistant and expert in co...,self_query_chain,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,<search_quality_reflection>\nThe web search re...
862,24,"I need a gluten-free pasta recipe, but please ...",0.1,2,1.0,3000,\nYou are a helpful assistant and expert in co...,self_query_chain,similarity,5,0.5,1,100,OpenAI,gpt-4o-mini,Here is a highly rated gluten-free pasta recip...


In [534]:
response_df

,Query_Question_No,Query_Question,Query_Response,BUCKET_NAME,BUCKET_NAME_TESTING,COARSE_LAMBDA,COARSE_SEARCH_TYPE,COARSE_TOP_K,DOWNLOAD_PATH,FILE_KEY,...,QDRANT_COLLECTION_NAME,QDRANT_SNAPSHOT_PATH,RERANKER_TOP_N,RETRIEVER,SELF_QUERY_API,SELF_QUERY_MODEL,LLM GateKeeper Judge,LLM GateKeeper Judge Reasoning,LLM Score,LLM Score Reasoning
Entry_1,1,I enjoy asian fusion food and I am a vegetaria...,"<search_quality_reflection>\nThe search for ""v...",recipes-rag,test-api-results,0.5,mmr,5,data,recipes_w_cleaning_time_combined_features.parquet,...,recipe_documents,qdrant/recipe_db_snapshot,1,self_query_chain,OpenAI,gpt-4o-mini,Yes,The recipe provided follows the user's request...,4,"[{'Accuracy': 4, 'Clarity': 4, 'Creativity': 4..."
Entry_2,2,I have a peanut allergy but I like thai food. ...,<search_quality_reflection>\nThe focused searc...,recipes-rag,test-api-results,0.5,mmr,5,data,recipes_w_cleaning_time_combined_features.parquet,...,recipe_documents,qdrant/recipe_db_snapshot,1,self_query_chain,OpenAI,gpt-4o-mini,Yes,The recipe generated by the other LLM appears ...,5,[{'Accuracy': 'The recipe is highly accurate a...


In [535]:
def llm_judge_eval(query_question, query_response):
    
    eval_message = f"""
    You are a helpful assistant and expert in reviewing cooking recipes.

    Please look at the given user query, recipe generated by another LLM and follow the rubric below. Provide a score between 1-5 on how good the recipe is.

    user query : "{query_question}"

    Recipe generated by another LLM: "{query_response}"

    Recipe review rubric: 

    Grading Scale (1-5)

    5 - Exceptional Recipe:

    1. Accuracy: The recipe is highly accurate and closely matches the user's query, including all specified ingredients, dietary restrictions, and desired cuisine type.
    2. Clarity: The instructions are clear, easy to follow, and logically sequenced. Cooking times and temperatures are precise.
    3. Creativity: The recipe demonstrates creativity, offering a unique or interesting twist on a classic dish or a novel combination of ingredients.
    4. Completeness: The recipe includes all necessary details, such as ingredient measurements, preparation steps, serving suggestions, and any relevant tips or variations.
    5. Healthiness: The recipe provides a balanced nutritional profile, aligning with any specified health goals or dietary considerations.
    6. User Feedback: The recipe is likely to receive high ratings from users for both taste and ease of preparation.
    4 - Very Good Recipe:

    1. Accuracy: The recipe mostly matches the user's query with minor deviations or substitutions that still align with the user's dietary restrictions and preferences.
    2. Clarity: The instructions are clear and easy to follow, with only minor areas that could benefit from additional detail.
    3. Creativity: The recipe shows some creativity and presents an appealing dish, though it may not be as unique as a 5-rated recipe.
    4. Completeness: The recipe includes most necessary details, but might miss a few minor tips or variations.
    5. Healthiness: The recipe is generally healthy, though it may not be as nutritionally balanced as a 5-rated recipe.
    6. User Feedback: The recipe is likely to receive good ratings from users, being tasty and reasonably easy to prepare.
    3 - Good Recipe:

    1. Accuracy: The recipe has a reasonable match with the user's query but may include some inaccuracies or ingredient substitutions that slightly alter the dish's nature.
    2. Clarity: The instructions are generally clear but may have a few confusing steps or lack detailed guidance in some areas.
    3. Creativity: The recipe is standard with minimal creativity or uniqueness.
    4. Completeness: The recipe includes the essential details but lacks additional helpful information or suggestions.
    5. Healthiness: The recipe is moderately healthy but may lack balance in terms of nutritional profile.
    6. User Feedback: The recipe is expected to receive average ratings, being satisfactory but not outstanding in taste or ease of preparation.
    2 - Fair Recipe:

    1. Accuracy: The recipe has noticeable discrepancies from the user's query, potentially including ingredients that were supposed to be excluded due to dietary restrictions.
    2. Clarity: The instructions are unclear or difficult to follow, with significant gaps or ambiguities.
    3. Creativity: The recipe lacks creativity and may appear bland or uninspired.
    4. Completeness: The recipe is missing several important details, such as precise measurements or key preparation steps.
    5. Healthiness: The recipe is not particularly healthy and may have an unbalanced nutritional profile.
    6. User Feedback: The recipe is likely to receive below-average ratings due to issues with taste, clarity, or preparation difficulty.
    1 - Poor Recipe:

    1. Accuracy: The recipe significantly deviates from the user's query, ignoring key dietary restrictions or preferences.
    2. Clarity: The instructions are confusing, incomplete, or incorrect, making the recipe difficult or impossible to follow.
    3. Creativity: The recipe is not creative and may seem haphazard or poorly thought out.
    4. Completeness: The recipe is missing critical details, such as major ingredients, steps, or cooking times.
    5. Healthiness: The recipe is unhealthy and lacks a balanced nutritional profile.
    6. User Feedback: The recipe is likely to receive low ratings due to poor taste, difficulty in preparation, or failure to meet user expectations.

    Return the output in JSON format with keys: "Score" (1-5), "Reasoning" (list of dicts with "Accuracy", "Clarity", "Creativity", "Completeness", "Healthiness", "User feedback").
    """

    eval_resp = qdrant_rag_chain_eval.invoke(eval_message)
    print(eval_resp)
    score  = int(eval_resp['content'][0]['text'].split()[2].split(",")[0])
    # reasoning = json.loads(eval_resp['content'][0]['text'])['Reasoning']
    # print(eval_resp['content'][0]['text'])
   
    return score
    

In [536]:
def llm_gatekeeper(query_question, query_response):

    eval_message = f"""
    You are a helpful assistant and expert in reviewing and judging cooking recipes. 
    Please be as unbiased and impartial towards the recipe.
    Please look at the given user query, recipe generated by another LLM. 
    Return if the response correctly follows the user's request, dietary restrictions and preferences. Answer, Yes or No.
    user query : "{query_question}"
    Recipe generated by another LLM: "{query_response}"
    Return the output in JSON format with keys: "Answer" (Yes or No), "Reasoning" (Provide one sentence reasoning).
    """

    eval_resp = qdrant_rag_chain_eval.invoke(eval_message)
    print(eval_resp)
    answer  = eval_resp['content'][0]['text'].split()[2].split(",")[0].replace('"', '')
    #reasoning = json.loads(json.loads(json.dumps(eval_resp))['content'][0]['text'])['Reasoning']
    # reasoning = json.loads(eval_resp['content'][0]['text'])['Reasoning']
    print(eval_resp['content'][0]['text'])

   
    return answer 


In [537]:
def append_llm_responses(df, is_gate_keeper = True):
    llm_scores = []
    llm_score_reasoning = []
    gate_keeper = []
    gate_keeper_reasoning = []
    for index, row in df.iterrows():

        # TODO : Change Test Query name and Test Query Response name
        query_question = row['Query_Question']
        query_response = row['Query_Response']
        if is_gate_keeper:
            gate_keeper_response = llm_gatekeeper(query_question, query_response)
            if gate_keeper_response == "Yes":
                gate_keeper_response = 1
            else:
                gate_keeper_response = 0
            gate_keeper.append(gate_keeper_response)
            # gate_keeper_reasoning.append(gate_keeper_response_reasoning)

        else:

            score= llm_judge_eval(query_question, query_response)
            llm_scores.append(score)
            # llm_score_reasoning.append(score_reasoning)
        

    # Append the LLM responses to the DataFrame
    if is_gate_keeper:
        df['LLM GateKeeper Judge'] = gate_keeper
        # df['LLM GateKeeper Judge Reasoning'] = gate_keeper_reasoning
    else:
        df['LLM Score'] = llm_scores
        # df['LLM Score Reasoning'] = llm_score_reasoning
    return df




In [538]:
# Call the function and get the updated DataFrame
gate_keeper_response = append_llm_responses(gate_keeper_receipe_df, is_gate_keeper=True)

{'id': 'msg_bdrk_012YFqtR4TW8QBP8gnkPFJTR', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-haiku-20240307', 'content': [{'type': 'text', 'text': '{\n    "Answer": "Yes",\n    "Reasoning": "The recipe provided is nut-free and meets the user\'s request for a dessert recipe that is suitable for their peanut allergy."\n}'}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 579, 'output_tokens': 51}}
{
    "Answer": "Yes",
    "Reasoning": "The recipe provided is nut-free and meets the user's request for a dessert recipe that is suitable for their peanut allergy."
}
{'id': 'msg_bdrk_01KpEqQrvgwqGLgHosgYVqm7', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-haiku-20240307', 'content': [{'type': 'text', 'text': '{\n    "Answer": "Yes",\n    "Reasoning": "The recipe generated by the other LLM correctly addresses the user\'s request for a dinner recipe that is suitable for their shellfish allergy."\n}'}], 'stop_reason': 'end_turn', 'stop_sequenc

In [539]:
normal_recipe_response = append_llm_responses(normal_recipe_df, is_gate_keeper=False)

{'id': 'msg_bdrk_01YGLjUsJSogqGHW2ShrGKnv', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-haiku-20240307', 'content': [{'type': 'text', 'text': '{\n    "Score": 5,\n    "Reasoning": [\n        {\n            "Accuracy": "The recipe closely matches the user\'s query, including all specified ingredients, dietary restrictions, and desired cuisine type.",\n            "Clarity": "The instructions are clear, easy to follow, and logically sequenced. Cooking times and temperatures are precise.",\n            "Creativity": "The recipe demonstrates creativity, offering a unique twist on a classic Szechuan stir-fry dish by incorporating vegetarian ingredients and Asian fusion flavors.",\n            "Completeness": "The recipe includes all necessary details, such as ingredient measurements, preparation steps, serving suggestions, and relevant tips or variations.",\n            "Healthiness": "The recipe provides a balanced nutritional profile, aligning with the user\'s vegetarian di

In [363]:
# dict_1 = {'id': 'msg_bdrk_01URZyeD5crwuzh5jcpEj12H', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-haiku-20240307', 'content': [{'type': 'text', 'text': '{\n    "Answer": "No",\n    "Reasoning": "The recipe provided is for a vegan seitan-based "chicken" substitute, which does not meet the user\'s request for a recipe with actual chicken."\n}'}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 780, 'output_tokens': 53}}


In [385]:
# dict_2 = {'id': 'msg_bdrk_01SPzvNNosmfdCD12DyW8MfC', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-haiku-20240307', 'content': [{'type': 'text', 'text': '{\n    "Score": 4,\n    "Reasoning": [\n        {\n            "Accuracy": 4,\n            "Clarity": 4,\n            "Creativity": 4,\n            "Completeness": 4,\n            "Healthiness": 4,\n            "User Feedback": 4\n        }\n    ]\n}\n\nExplanation:\nThe recipe generated by the other LLM is a very good match for the user\'s query. It includes all the key ingredients and follows the user\'s preference for a vegetarian Asian fusion dish. The instructions are clear and logically sequenced, making it easy to follow. While the recipe doesn\'t demonstrate exceptional creativity, it does offer an appealing and unique combination of flavors. The recipe includes all the necessary details, and the nutritional profile seems generally healthy, though it could be further optimized. Overall, this recipe is likely to receive good ratings from users for both taste and ease of preparation.'}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 1498, 'output_tokens': 214}}




In [560]:
dict_2 = {'id': 'msg_bdrk_01SPzvNNosmfdCD12DyW8MfC', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-haiku-20240307', 'content': [{'type': 'text', 'text': '{\n    "Score": 4,\n    "Reasoning": [\n        {\n            "Accuracy": 4,\n            "Clarity": 4,\n            "Creativity": 4,\n            "Completeness": 4,\n            "Healthiness": 4,\n            "User Feedback": 4\n        }\n    ]\n}\n\nExplanation:\nThe recipe generated by the other LLM is a very good match for the user\'s query. It includes all the key ingredients and follows the user\'s preference for a vegetarian Asian fusion dish. The instructions are clear and logically sequenced, making it easy to follow. While the recipe doesn\'t demonstrate exceptional creativity, it does offer an appealing and unique combination of flavors. The recipe includes all the necessary details, and the nutritional profile seems generally healthy, though it could be further optimized. Overall, this recipe is likely to receive good ratings from users for both taste and ease of preparation.'}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 1498, 'output_tokens': 214}}




In [552]:
min(normal_recipe_response['LLM Score'])

4

In [565]:
normal_recipe_response.groupby(["retriever"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/1223432679.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  normal_recipe_response.groupby(["retriever"]).mean()


,Query_Question_No,top_p,top_k,temperature,max_tokens,coarse_top_k,coarse_lambda,reranker_top_n,coarse_fetch_k,LLM Score
retriever,,,,,,,,,,
coarse,8.933333,0.5,6.0,0.533333,3000.0,5.0,0.5,1.0,100.0,4.866667
reranker,8.933333,0.5,6.0,0.533333,3000.0,5.0,0.5,1.0,100.0,4.711111
self_query_chain,8.933333,0.5,6.0,0.533333,3000.0,5.0,0.5,1.0,100.0,4.666667


In [555]:
normal_recipe_response.groupby(["temperature"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/1787859334.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  normal_recipe_response.groupby(["temperature"]).mean()


,Query_Question_No,top_p,top_k,max_tokens,coarse_top_k,coarse_lambda,reranker_top_n,coarse_fetch_k,LLM Score
temperature,,,,,,,,,
0.1,8.933333,0.5,6.0,3000.0,5.0,0.5,1.0,100.0,4.783333
0.5,8.933333,0.5,6.0,3000.0,5.0,0.5,1.0,100.0,4.755556
1.0,8.933333,0.5,6.0,3000.0,5.0,0.5,1.0,100.0,4.705556


In [566]:
normal_recipe_response.groupby(["top_p"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/3656170978.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  normal_recipe_response.groupby(["top_p"]).mean()


,Query_Question_No,top_k,temperature,max_tokens,coarse_top_k,coarse_lambda,reranker_top_n,coarse_fetch_k,LLM Score
top_p,,,,,,,,,
0.1,8.933333,6.0,0.533333,3000.0,5.0,0.5,1.0,100.0,4.762963
0.9,8.933333,6.0,0.533333,3000.0,5.0,0.5,1.0,100.0,4.733333


In [567]:
normal_recipe_response.groupby(["top_k"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/2934910060.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  normal_recipe_response.groupby(["top_k"]).mean()


,Query_Question_No,top_p,temperature,max_tokens,coarse_top_k,coarse_lambda,reranker_top_n,coarse_fetch_k,LLM Score
top_k,,,,,,,,,
2,8.933333,0.5,0.533333,3000.0,5.0,0.5,1.0,100.0,4.751852
10,8.933333,0.5,0.533333,3000.0,5.0,0.5,1.0,100.0,4.744444


In [568]:
normal_recipe_response.groupby(["retriever", "temperature"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/2850549410.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  normal_recipe_response.groupby(["retriever", "temperature"]).mean()


Query_Question_No  top_p  top_k  max_tokens  \
retriever        temperature                                                
coarse           0.1                   8.933333    0.5    6.0      3000.0   
                 0.5                   8.933333    0.5    6.0      3000.0   
                 1.0                   8.933333    0.5    6.0      3000.0   
reranker         0.1                   8.933333    0.5    6.0      3000.0   
                 0.5                   8.933333    0.5    6.0      3000.0   
                 1.0                   8.933333    0.5    6.0      3000.0   
self_query_chain 0.1                   8.933333    0.5    6.0      3000.0   
                 0.5                   8.933333    0.5    6.0      3000.0   
                 1.0                   8.933333    0.5    6.0      3000.0   

                              coarse_top_k  coarse_lambda  reranker_top_n  \
retriever        temperature                                                
coarse           0.1                   5.0            0.5             1.0   
                 0.5                   5.0            0.5             1.0   
                 1.0                   5.0            0.5             1.0   
reranker         0.1                   5.0            0.5             1.0   
                 0.5                   5.0            0.5             1.0   
                 1.0                   5.0            0.5             1.0   
self_query_chain 0.1                   5.0            0.5             1.0   
                 0.5                   5.0            0.5             1.0   
                 1.0                   5.0            0.5             1.0   

                              coarse_fetch_k  LLM Score  
retriever        temperature                             
coarse           0.1                   100.0   4.866667  
                 0.5                   100.0   4.900000  
                 1.0                   100.0   4.833333  
reranker         0.1                   100.0   4.733333  
                 0.5                   100.0   4.716667  
                 1.0                   100.0   4.683333  
self_query_chain 0.1                   100.0   4.750000  
                 0.5                   100.0   4.650000  
                 1.0                   100.0   4.600000

In [569]:
normal_recipe_response.groupby(["retriever", "temperature","top_p"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/2891312367.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  normal_recipe_response.groupby(["retriever", "temperature","top_p"]).mean()


Query_Question_No  top_k  max_tokens  \
retriever        temperature top_p                                         
coarse           0.1         0.1             8.933333    6.0      3000.0   
                             0.9             8.933333    6.0      3000.0   
                 0.5         0.1             8.933333    6.0      3000.0   
                             0.9             8.933333    6.0      3000.0   
                 1.0         0.1             8.933333    6.0      3000.0   
                             0.9             8.933333    6.0      3000.0   
reranker         0.1         0.1             8.933333    6.0      3000.0   
                             0.9             8.933333    6.0      3000.0   
                 0.5         0.1             8.933333    6.0      3000.0   
                             0.9             8.933333    6.0      3000.0   
                 1.0         0.1             8.933333    6.0      3000.0   
                             0.9             8.933333    6.0      3000.0   
self_query_chain 0.1         0.1             8.933333    6.0      3000.0   
                             0.9             8.933333    6.0      3000.0   
                 0.5         0.1             8.933333    6.0      3000.0   
                             0.9             8.933333    6.0      3000.0   
                 1.0         0.1             8.933333    6.0      3000.0   
                             0.9             8.933333    6.0      3000.0   

                                    coarse_top_k  coarse_lambda  \
retriever        temperature top_p                                
coarse           0.1         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 0.5         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 1.0         0.1             5.0            0.5   
                             0.9             5.0            0.5   
reranker         0.1         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 0.5         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 1.0         0.1             5.0            0.5   
                             0.9             5.0            0.5   
self_query_chain 0.1         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 0.5         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 1.0         0.1             5.0            0.5   
                             0.9             5.0            0.5   

                                    reranker_top_n  coarse_fetch_k  LLM Score  
retriever        temperature top_p                                             
coarse           0.1         0.1               1.0           100.0   4.866667  
                             0.9               1.0           100.0   4.866667  
                 0.5         0.1               1.0           100.0   4.900000  
                             0.9               1.0           100.0   4.900000  
                 1.0         0.1               1.0           100.0   4.900000  
                             0.9               1.0           100.0   4.766667  
reranker         0.1         0.1               1.0           100.0   4.733333  
                             0.9               1.0           100.0   4.733333  
                 0.5         0.1               1.0           100.0   4.733333  
                             0.9               1.0           100.0   4.700000  
                 1.0         0.1               1.0           100.0   4.700000  
                             0.9               1.0           100.0   4.666667  
self_query_chain 0.1         0.1               1.0    

In [563]:
normal_recipe_response.groupby(["retriever", "temperature","top_p", "top_k"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/1929535888.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  normal_recipe_response.groupby(["retriever", "temperature","top_p", "top_k"]).mean()


Query_Question_No  max_tokens  \
retriever        temperature top_p top_k                                  
coarse           0.1         0.1   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                             0.9   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                 0.5         0.1   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                             0.9   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                 1.0         0.1   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                             0.9   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
reranker         0.1         0.1   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                             0.9   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                 0.5         0.1   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                             0.9   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                 1.0         0.1   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                             0.9   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
self_query_chain 0.1         0.1   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                             0.9   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                 0.5         0.1   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                             0.9   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                 1.0         0.1   2               8.933333      3000.0   
                                   10              8.933333      3000.0   
                             0.9   2               8.933333      3000.0   
                                   10              8.933333      3000.0   

                                          coarse_top_k  coarse_lambda  \
retriever        temperature top_p top_k                                
coarse           0.1         0.1   2               5.0            0.5   
                                   10              5.0            0.5   
                             0.9   2               5.0            0.5   
                                   10              5.0            0.5   
                 0.5         0.1   2               5.0            0.5   
                                   10              5.0            0.5   
                             0.9   2               5.0            0.5   
                                   10              5.0            0.5   
                 1.0         0.1   2               5.0            0.5   
                                   10              5.0            0.5   
                             0.9   2               5.0            0.5   
                                   10              5.0            0.5   
reranker         0.1         0.1   2               5.0            0.5   
                                   10              5.0            0.5   
                       

In [553]:
gate_keeper_response.groupby(["retriever"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/1816825482.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gate_keeper_response.groupby(["retriever"]).mean()


,Query_Question_No,top_p,top_k,temperature,max_tokens,coarse_top_k,coarse_lambda,reranker_top_n,coarse_fetch_k,LLM GateKeeper Judge
retriever,,,,,,,,,,
coarse,13.458333,0.5,6.0,0.533333,3000.0,5.0,0.5,1.0,100.0,0.937500
reranker,13.458333,0.5,6.0,0.533333,3000.0,5.0,0.5,1.0,100.0,0.895833
self_query_chain,13.458333,0.5,6.0,0.533333,3000.0,5.0,0.5,1.0,100.0,0.913194


In [554]:
gate_keeper_response.groupby(["temperature"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/1762499044.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gate_keeper_response.groupby(["temperature"]).mean()


,Query_Question_No,top_p,top_k,max_tokens,coarse_top_k,coarse_lambda,reranker_top_n,coarse_fetch_k,LLM GateKeeper Judge
temperature,,,,,,,,,
0.1,13.458333,0.5,6.0,3000.0,5.0,0.5,1.0,100.0,0.902778
0.5,13.458333,0.5,6.0,3000.0,5.0,0.5,1.0,100.0,0.920139
1.0,13.458333,0.5,6.0,3000.0,5.0,0.5,1.0,100.0,0.923611


In [558]:
gate_keeper_response.groupby(["top_p"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/2687672870.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gate_keeper_response.groupby(["top_p"]).mean()


,Query_Question_No,top_k,temperature,max_tokens,coarse_top_k,coarse_lambda,reranker_top_n,coarse_fetch_k,LLM GateKeeper Judge
top_p,,,,,,,,,
0.1,13.458333,6.0,0.533333,3000.0,5.0,0.5,1.0,100.0,0.907407
0.9,13.458333,6.0,0.533333,3000.0,5.0,0.5,1.0,100.0,0.923611


In [559]:
gate_keeper_response.groupby(["top_k"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/3503906397.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gate_keeper_response.groupby(["top_k"]).mean()


,Query_Question_No,top_p,temperature,max_tokens,coarse_top_k,coarse_lambda,reranker_top_n,coarse_fetch_k,LLM GateKeeper Judge
top_k,,,,,,,,,
2,13.458333,0.5,0.533333,3000.0,5.0,0.5,1.0,100.0,0.907407
10,13.458333,0.5,0.533333,3000.0,5.0,0.5,1.0,100.0,0.923611


In [570]:
gate_keeper_response.groupby(["retriever", "temperature"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/2442311540.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gate_keeper_response.groupby(["retriever", "temperature"]).mean()


Query_Question_No  top_p  top_k  max_tokens  \
retriever        temperature                                                
coarse           0.1                  13.458333    0.5    6.0      3000.0   
                 0.5                  13.458333    0.5    6.0      3000.0   
                 1.0                  13.458333    0.5    6.0      3000.0   
reranker         0.1                  13.458333    0.5    6.0      3000.0   
                 0.5                  13.458333    0.5    6.0      3000.0   
                 1.0                  13.458333    0.5    6.0      3000.0   
self_query_chain 0.1                  13.458333    0.5    6.0      3000.0   
                 0.5                  13.458333    0.5    6.0      3000.0   
                 1.0                  13.458333    0.5    6.0      3000.0   

                              coarse_top_k  coarse_lambda  reranker_top_n  \
retriever        temperature                                                
coarse           0.1                   5.0            0.5             1.0   
                 0.5                   5.0            0.5             1.0   
                 1.0                   5.0            0.5             1.0   
reranker         0.1                   5.0            0.5             1.0   
                 0.5                   5.0            0.5             1.0   
                 1.0                   5.0            0.5             1.0   
self_query_chain 0.1                   5.0            0.5             1.0   
                 0.5                   5.0            0.5             1.0   
                 1.0                   5.0            0.5             1.0   

                              coarse_fetch_k  LLM GateKeeper Judge  
retriever        temperature                                        
coarse           0.1                   100.0              0.937500  
                 0.5                   100.0              0.937500  
                 1.0                   100.0              0.937500  
reranker         0.1                   100.0              0.854167  
                 0.5                   100.0              0.916667  
                 1.0                   100.0              0.916667  
self_query_chain 0.1                   100.0              0.916667  
                 0.5                   100.0              0.906250  
                 1.0                   100.0              0.916667

In [571]:
gate_keeper_response.groupby(["retriever", "temperature", "top_p"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/2325129966.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gate_keeper_response.groupby(["retriever", "temperature", "top_p"]).mean()


Query_Question_No  top_k  max_tokens  \
retriever        temperature top_p                                         
coarse           0.1         0.1            13.458333    6.0      3000.0   
                             0.9            13.458333    6.0      3000.0   
                 0.5         0.1            13.458333    6.0      3000.0   
                             0.9            13.458333    6.0      3000.0   
                 1.0         0.1            13.458333    6.0      3000.0   
                             0.9            13.458333    6.0      3000.0   
reranker         0.1         0.1            13.458333    6.0      3000.0   
                             0.9            13.458333    6.0      3000.0   
                 0.5         0.1            13.458333    6.0      3000.0   
                             0.9            13.458333    6.0      3000.0   
                 1.0         0.1            13.458333    6.0      3000.0   
                             0.9            13.458333    6.0      3000.0   
self_query_chain 0.1         0.1            13.458333    6.0      3000.0   
                             0.9            13.458333    6.0      3000.0   
                 0.5         0.1            13.458333    6.0      3000.0   
                             0.9            13.458333    6.0      3000.0   
                 1.0         0.1            13.458333    6.0      3000.0   
                             0.9            13.458333    6.0      3000.0   

                                    coarse_top_k  coarse_lambda  \
retriever        temperature top_p                                
coarse           0.1         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 0.5         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 1.0         0.1             5.0            0.5   
                             0.9             5.0            0.5   
reranker         0.1         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 0.5         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 1.0         0.1             5.0            0.5   
                             0.9             5.0            0.5   
self_query_chain 0.1         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 0.5         0.1             5.0            0.5   
                             0.9             5.0            0.5   
                 1.0         0.1             5.0            0.5   
                             0.9             5.0            0.5   

                                    reranker_top_n  coarse_fetch_k  \
retriever        temperature top_p                                   
coarse           0.1         0.1               1.0           100.0   
                             0.9               1.0           100.0   
                 0.5         0.1               1.0           100.0   
                             0.9               1.0           100.0   
                 1.0         0.1               1.0           100.0   
                             0.9               1.0           100.0   
reranker         0.1         0.1               1.0           100.0   
                             0.9               1.0           100.0   
                 0.5         0.1               1.0           100.0   
                             0.9               1.0           100.0   
                 1.0         0.1               1.0           100.0   
                             0.9               1.0           100.0   
self_query_chain 0.1         0.1               1.0           100.0   
                             0.9               1.0           100.0   
                 0.5         0.1               1.0    

In [564]:
gate_keeper_response.groupby(["retriever", "temperature","top_p", "top_k"]).mean()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/3043594381.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gate_keeper_response.groupby(["retriever", "temperature","top_p", "top_k"]).mean()


Query_Question_No  max_tokens  \
retriever        temperature top_p top_k                                  
coarse           0.1         0.1   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                             0.9   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                 0.5         0.1   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                             0.9   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                 1.0         0.1   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                             0.9   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
reranker         0.1         0.1   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                             0.9   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                 0.5         0.1   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                             0.9   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                 1.0         0.1   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                             0.9   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
self_query_chain 0.1         0.1   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                             0.9   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                 0.5         0.1   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                             0.9   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                 1.0         0.1   2              13.458333      3000.0   
                                   10             13.458333      3000.0   
                             0.9   2              13.458333      3000.0   
                                   10             13.458333      3000.0   

                                          coarse_top_k  coarse_lambda  \
retriever        temperature top_p top_k                                
coarse           0.1         0.1   2               5.0            0.5   
                                   10              5.0            0.5   
                             0.9   2               5.0            0.5   
                                   10              5.0            0.5   
                 0.5         0.1   2               5.0            0.5   
                                   10              5.0            0.5   
                             0.9   2               5.0            0.5   
                                   10              5.0            0.5   
                 1.0         0.1   2               5.0            0.5   
                                   10              5.0            0.5   
                             0.9   2               5.0            0.5   
                                   10              5.0            0.5   
reranker         0.1         0.1   2               5.0            0.5   
                                   10              5.0            0.5   
                       

In [573]:
gate_keeper_score = gate_keeper_response.groupby(["retriever", "temperature","top_p", "top_k"]).mean().reset_index()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/1998933866.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  gate_keeper_score = gate_keeper_response.groupby(["retriever", "temperature","top_p", "top_k"]).mean().reset_index()


In [574]:
normal_recipe_score = normal_recipe_df.groupby(["retriever", "temperature","top_p", "top_k"]).mean().reset_index()

/var/folders/kg/_m6vf_452x3cp68n6hz24vwm0000gn/T/ipykernel_97043/2796516199.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  normal_recipe_score = normal_recipe_df.groupby(["retriever", "temperature","top_p", "top_k"]).mean().reset_index()


In [578]:
gate_keeper_score.drop(["Query_Question_No", "max_tokens", "coarse_top_k", "coarse_lambda", "reranker_top_n", "coarse_fetch_k"], axis=1, inplace=True)

In [579]:
normal_recipe_score.drop(["Query_Question_No", "max_tokens", "coarse_top_k", "coarse_lambda", "reranker_top_n", "coarse_fetch_k"], axis=1, inplace=True)

In [585]:
final_df = gate_keeper_score.merge(normal_recipe_score,  how='inner', on=['temperature', "top_p", "top_k", "retriever"])

In [591]:
final_df['Total_Score'] = (final_df['LLM GateKeeper Judge'] + final_df['LLM Score'])/2

In [594]:
final_df.sort_values(by=['Total_Score'], ascending=False).head(10)

,retriever,temperature,top_p,top_k,LLM GateKeeper Judge,LLM Score,Total_Score
2,coarse,0.1,0.9,2,0.916667,5.000000,2.958333
6,coarse,0.5,0.9,2,0.958333,4.933333,2.945833
1,coarse,0.1,0.1,10,0.958333,4.933333,2.945833
5,coarse,0.5,0.1,10,0.916667,4.933333,2.925000
8,coarse,1.0,0.1,2,0.916667,4.933333,2.925000
25,self_query_chain,0.1,0.1,10,0.958333,4.866667,2.912500
4,coarse,0.5,0.1,2,0.958333,4.866667,2.912500
7,coarse,0.5,0.9,10,0.916667,4.866667,2.891667
9,coarse,1.0,0.1,10,0.916667,4.866667,2.891667
27,self_query_chain,0.1,0.9,10,0.916667,4.866667,2.891667


In [550]:
normal_recipe_response.to_csv("normal_recipe_response_llm_score.csv")

In [551]:
gate_keeper_response.to_csv("gatekeeper_response_llm_score.csv")